In [1]:
from bs4 import BeautifulSoup
import requests
import re
from pymongo import MongoClient

In [2]:
APARTMENT = 'Ava Pacific Beach'

url = 'https://api.avalonbay.com/json/reply/ApartmentSearch?communityCode=CA025&min=1600&max=2800&_=1550548620970'
r = requests.get(url, timeout=5).json()

results = []

available_floorplans = r['results']['availableFloorPlanTypes']
two_bedroom = [x for x in available_floorplans if x['floorPlanTypeCode'] == '2BD'][0]['availableFloorPlans']
for fp in two_bedroom:

    for a in fp['finishPackages'][0]['apartments']:

        data = {}

        data['apartment'] = APARTMENT
        data['floorplan'] = 'TODO'
        data['unit'] = a['apartmentNumber']
        data['floor'] = a['floor']

        apartment_code = a['apartmentCode']
        url = 'https://www.avaloncommunities.com/california/san-diego-apartments/ava-pacific-beach/apartment/' + apartment_code

        r2 = requests.get(url, timeout=5)
        soup = BeautifulSoup(r2.content, "html.parser")

        leases = soup.find('div', {'class': 'terms'}).select('table')[0].findAll('tr')
        for lease in leases:
            
            td_elements = lease.select('td')
            
            length = re.sub('[^0-9]', '', td_elements[0].text)  # regex to remove all non-digits
            price = int(re.sub('[^0-9]', '',  td_elements[-1].text)) # regex to remove all non-digits

            data['lease_' + length] = price


        results.append(data)

results

[{'apartment': 'Ava Pacific Beach',
  'floorplan': 'TODO',
  'unit': '00C-C111',
  'floor': 1,
  'lease_12': 2585,
  'lease_11': 2590,
  'lease_10': 2455,
  'lease_9': 3190,
  'lease_8': 3150,
  'lease_7': 3520,
  'lease_6': 3230,
  'lease_5': 4085,
  'lease_4': 3365,
  'lease_3': 3400,
  'lease_2': 3445},
 {'apartment': 'Ava Pacific Beach',
  'floorplan': 'TODO',
  'unit': '00S-S307',
  'floor': 3,
  'lease_12': 2600,
  'lease_11': 2605,
  'lease_10': 2470,
  'lease_9': 3205,
  'lease_8': 3165,
  'lease_7': 3535,
  'lease_6': 3245,
  'lease_5': 4100,
  'lease_4': 3380,
  'lease_3': 3415,
  'lease_2': 3460},
 {'apartment': 'Ava Pacific Beach',
  'floorplan': 'TODO',
  'unit': '00V-V206',
  'floor': 2,
  'lease_12': 2700,
  'lease_11': 2705,
  'lease_10': 2570,
  'lease_9': 3305,
  'lease_8': 3265,
  'lease_7': 3635,
  'lease_6': 3345,
  'lease_5': 4200,
  'lease_4': 3480,
  'lease_3': 3515,
  'lease_2': 3560},
 {'apartment': 'Ava Pacific Beach',
  'floorplan': 'TODO',
  'unit': '00V-V1

In [3]:
client = MongoClient('db', 27017, username='apartments', password='apartments')

db_units = client.apartments.units

db_result = db_units.insert_many(results)
print('Multiple posts: {0}'.format(db_result.inserted_ids))

Multiple posts: [ObjectId('5c6cdf13199a99009798338d'), ObjectId('5c6cdf13199a99009798338e'), ObjectId('5c6cdf13199a99009798338f'), ObjectId('5c6cdf13199a990097983390'), ObjectId('5c6cdf13199a990097983391'), ObjectId('5c6cdf13199a990097983392'), ObjectId('5c6cdf13199a990097983393'), ObjectId('5c6cdf13199a990097983394'), ObjectId('5c6cdf13199a990097983395'), ObjectId('5c6cdf13199a990097983396'), ObjectId('5c6cdf13199a990097983397'), ObjectId('5c6cdf13199a990097983398'), ObjectId('5c6cdf13199a990097983399'), ObjectId('5c6cdf13199a99009798339a'), ObjectId('5c6cdf13199a99009798339b'), ObjectId('5c6cdf13199a99009798339c'), ObjectId('5c6cdf13199a99009798339d'), ObjectId('5c6cdf13199a99009798339e'), ObjectId('5c6cdf13199a99009798339f'), ObjectId('5c6cdf13199a9900979833a0')]
